## Plan to focus on Cell Processes {2 members} - not good not done
- [ ] EG11105  --> proofreading gene, skip for now.
- [ ] G6932  --> true dead-end reached. skip for now.

- [ ] EG10722
- [ ] G7841

In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import dill
import cvxpy as cp

%matplotlib inline
# sns.set(style='darkgrid', palette='viridis', context='talk')

os.chdir(os.path.expanduser('~/dev/vivarium-ecoli'))

from ecoli.processes.metabolism_redux_classic import NetworkFlowModel, FlowResult
from ecoli.processes.registries import topology_registry
TOPOLOGY = topology_registry.access("ecoli-metabolism-redux")

In [2]:
# load basal new model
time = '400'
date = '2025-01-30'
experiment = 'NEW_updated_bad_rxn'
condition = 'basal'
entry = f'{experiment}_{time}_{date}'
folder = f'out/metabolism-comparison/{condition}/{entry}/'

output = np.load(folder + '0_output.npy',allow_pickle='TRUE').item()
# output = np.load(r"out/geneRxnVerifData/output_glc.npy", allow_pickle=True, encoding='ASCII').tolist()
output = output['agents']['0']
fba_basal = output['listeners']['fba_results']
bulk = pd.DataFrame(output['bulk'])
f = open(folder + 'agent_steps.pkl', 'rb')
agent_basal = dill.load(f)
f.close()

In [3]:
# get commonly stored variables
metabolism_basal = agent_basal['ecoli-metabolism-redux-classic']
stoichiometry = metabolism_basal.stoichiometry.copy()
reaction_names = metabolism_basal.reaction_names
kinetic_reaction_ids = metabolism_basal.kinetic_constraint_reactions
fba_new_reaction_ids = metabolism_basal.parameters["fba_new_reaction_ids"]
fba_reaction_ids_to_base_reaction_ids = metabolism_basal.parameters['fba_reaction_ids_to_base_reaction_ids']
binary_kinetic_idx = metabolism_basal.binary_kinetic_idx
metabolites = metabolism_basal.metabolite_names

S = stoichiometry.copy()
S = pd.DataFrame(S, index=metabolites, columns=reaction_names)
homeostatic_count = pd.DataFrame(fba_basal["homeostatic_metabolite_counts"], columns=metabolism_basal.homeostatic_metabolites).loc[24, :].copy()
homeostatic = pd.DataFrame(fba_basal["target_homeostatic_dmdt"], columns=metabolism_basal.homeostatic_metabolites).loc[24, :].copy()
maintenance = pd.DataFrame(fba_basal["maintenance_target"][1:], columns=['maintenance_reaction']).iat[24, 0]

FREE_RXNS = ["TRANS-RXN-145", "TRANS-RXN0-545", "TRANS-RXN0-474"]

# load variables from track_reaction_usage.ipynb

In [4]:
%store -r df_reaction_flux df_reaction_track df_gene_usage unused_kinetic_gene

In [5]:
def get_subset_S(S, met_of_interest):
    S_met = S.loc[met_of_interest, :]
    S_met = S_met.loc[:,~np.all(S_met == 0, axis=0)]
    return S_met, S_met.columns

def test_NetworkFlowModel(objective_weights,
                          uptake_addition = set([]), uptake_removal = set([]), new_exchange_molecules = set([]),
                          add_metabolite = None, add_reaction = None, add_kinetic = None, remove_reaction = None, 
                          add_homeostatic_demand = None, force_reaction = None):
    # update exchanges
    uptake = metabolism_basal.allowed_exchange_uptake.copy()
    uptake = set(uptake)
    uptake = uptake | uptake_addition
    uptake = uptake - uptake_removal
    
    exchange_molecules = metabolism_basal.exchange_molecules.copy()
    exchange_molecules = exchange_molecules | new_exchange_molecules
    
    # update stoichiometry
    reaction_names = metabolism_basal.reaction_names.copy()
    kinetic_reaction_ids = metabolism_basal.kinetic_constraint_reactions.copy()
    kinetic = pd.DataFrame(fba_basal["target_kinetic_fluxes"], columns=metabolism_basal.kinetic_constraint_reactions).loc[24, :].copy()
    metabolites = metabolism_basal.metabolite_names.copy()
    homeostatic = pd.DataFrame(fba_basal["target_homeostatic_dmdt"], columns=metabolism_basal.homeostatic_metabolites).loc[24, :].copy()
    homeostatic_counts = homeostatic_count.copy() * metabolism_basal.counts_to_molar.asNumber()
    
    S_new = stoichiometry.copy()
    
    if add_metabolite is not None: # add to metabolites list because they are currently not included in the model
        for m in add_metabolite:
            if m not in metabolites:
                metabolites.append(m)
        # append rows of zeros to S_new of length add_metabolite
        S_new = np.concatenate((S_new, np.zeros((len(add_metabolite), S_new.shape[1]))), axis=0)
        print(S_new.shape)
    
    if add_reaction is not None:
        # assert add_reaction is a dictionary
        assert isinstance(add_reaction, dict)
        
        for r,s in add_reaction.items():
            if r not in reaction_names:
                reaction_names.append(r)
            # append columns of reaction stoich to S_new of length add_reaction
            new_reaction = np.zeros((S_new.shape[0], 1))
            for m, v in s.items():
                new_reaction[metabolites.index(m), 0] = v
            S_new = np.concatenate((S_new, new_reaction), axis=1)
        print(S_new.shape)
    
    if add_kinetic is not None:
        # assert add_kinetic is a dictionary
        assert isinstance(add_kinetic, dict)
        
        for r, v in add_kinetic.items():
            if r not in kinetic_reaction_ids:
                kinetic_reaction_ids.append(r)
                kinetic[r] = v
                
    if remove_reaction is not None:
        for r in remove_reaction:
            r_idx = reaction_names.index(r)
            S_new = np.delete(S_new, r_idx, axis=1)
            reaction_names.remove(r)
            if r in kinetic_reaction_ids:
                kinetic_reaction_ids.remove(r)
                del kinetic[r]
                
    if force_reaction is not None:
        force_reaction_idx = np.array([reaction_names.index(r) for r in force_reaction])
    else:
        force_reaction_idx = force_reaction

    if add_homeostatic_demand is not None:
        # assert add_homeostatic_demand is a set
        assert isinstance(add_homeostatic_demand, list)
        
        for met in add_homeostatic_demand:
            homeostatic[met] = 100
            homeostatic_counts[met] = 1
            
    print(f'Force Reaction ID is:',force_reaction_idx)
    # Solve NetworkFlowModel
    model = NetworkFlowModel(
            stoich_arr=S_new,
            metabolites=metabolites,
            reactions=reaction_names,
            homeostatic_metabolites=list(dict(homeostatic).keys()),
            kinetic_reactions=kinetic_reaction_ids,
            free_reactions=FREE_RXNS)
    model.set_up_exchanges(exchanges=exchange_molecules, uptakes=uptake)
    solution: FlowResult = model.solve(
            homeostatic_concs=homeostatic_counts, # in conc
            homeostatic_dm_targets=np.array(list(dict(homeostatic).values())), # *10^7
            maintenance_target=maintenance, # *10^6 ish
            kinetic_targets=np.array(list(dict(kinetic).values())), # *10^6 ish
            # binary_kinetic_idx=binary_kinetic_idx, #7646
            binary_kinetic_idx=None,
            force_flow_idx=force_reaction_idx,
            objective_weights=objective_weights, #same
            upper_flux_bound= 1000000000, # increase to 10^9 because notebook runs FlowResult using Counts, WC runs using conc.   
            solver=cp.GLOP) #SCS. ECOS
    print(f'Optimal objective function reached is: {solution.objective}')
    return solution.velocities, reaction_names, S_new, metabolites, kinetic

# EG11105  --> entH --> RXN0-7105 and RXN0-7046

Fix RXN0-7105. Part of enterobactin biosynthesis pathway.

# G6932 --> hxpB --> SORBITOL-6-PHOSPHATASE-RXN, MANNITOL-1-PHOSPHATASE-RXN, 3.1.3.68-RXN

In [18]:
met_of_interest = ['D-SORBITOL-6-P[c]', 'SORBITOL[c]']
S_met, rxn = get_subset_S(S, met_of_interest)
df_reaction_flux.loc[rxn,:]

,basal,acetate,rich,anaerobic_basal,anaerobic_acetate,anaerobic_rich,is_new,is_kinetic
RXN0-7189-D-SORBITOL-6-P//D-SORBITOL-6-P.31.,0.0,0.0,0.0,-0.0,-0.0,False,False,False
RXN0-7189-D-SORBITOL-6-P//D-SORBITOL-6-P.31. (reverse),0.0,0.0,0.0,-0.0,-0.0,False,False,False
RXN0-7189-SORBITOL//SORBITOL.19.,0.0,0.0,0.0,-0.0,-0.0,False,False,False
RXN0-7189-SORBITOL//SORBITOL.19. (reverse),0.0,0.0,0.0,-0.0,-0.0,False,False,False
SORB6PDEHYDROG-RXN,0.0,0.0,0.0,-0.0,-0.0,False,False,False
SORB6PDEHYDROG-RXN (reverse),0.0,0.0,0.0,-0.0,0.0,False,False,False
SORB6PDEHYDROG-RXN-D-SORBITOL-6-P/NAD//CPD-18720/NADH/PROTON.42.,0.0,0.0,0.0,0.0,-0.0,False,False,False
SORB6PDEHYDROG-RXN-D-SORBITOL-6-P/NAD//CPD-18720/NADH/PROTON.42. (reverse),0.0,0.0,0.0,-0.0,-0.0,False,False,False
SORB6PDEHYDROG-RXN-D-SORBITOL-6-P/NAD//FRUCTOSE-6P/NADH/PROTON.44.,0.0,0.0,0.0,-0.0,-0.0,False,False,False
SORB6PDEHYDROG-RXN-D-SORBITOL-6-P/NAD//FRUCTOSE-6P/NADH/PROTON.44. (reverse),0.0,0.0,0.0,-0.0,-0.0,False,False,False


In [16]:
rxn = ['SORBITOL-6-PHOSPHATASE-RXN']
temp = S.loc[:,rxn]
temp = temp.loc[~np.any(temp==0, axis=1), :]
temp

,SORBITOL-6-PHOSPHATASE-RXN
D-SORBITOL-6-P[c],-1
Pi[c],1
SORBITOL[c],1
WATER[c],-1


In [20]:
S_met.T

,D-SORBITOL-6-P[c],SORBITOL[c]
RXN0-7189-D-SORBITOL-6-P//D-SORBITOL-6-P.31.,1,0
RXN0-7189-D-SORBITOL-6-P//D-SORBITOL-6-P.31. (reverse),-1,0
RXN0-7189-SORBITOL//SORBITOL.19.,0,1
RXN0-7189-SORBITOL//SORBITOL.19. (reverse),0,-1
SORB6PDEHYDROG-RXN,-1,0
SORB6PDEHYDROG-RXN (reverse),1,0
SORB6PDEHYDROG-RXN-D-SORBITOL-6-P/NAD//CPD-18720/NADH/PROTON.42.,-1,0
SORB6PDEHYDROG-RXN-D-SORBITOL-6-P/NAD//CPD-18720/NADH/PROTON.42. (reverse),1,0
SORB6PDEHYDROG-RXN-D-SORBITOL-6-P/NAD//FRUCTOSE-6P/NADH/PROTON.44.,-1,0
SORB6PDEHYDROG-RXN-D-SORBITOL-6-P/NAD//FRUCTOSE-6P/NADH/PROTON.44. (reverse),1,0


## try allowing FRUCTOSE-6P[c] uptake

In [32]:
uptake_addition = set([
                       'CPD-18720[e]',
                       ])
uptake_removal = set([
                      # "OXYGEN-MOLECULE[p]",
                      'GLC[p]',
                     ])
add_reaction = {}
remove_reaction = ['RXN-14812-CPD-18720//FRUCTOSE-6P.23.']
# new_exchange_molecules = set(['23-Diaminopropanoate[e]']) 

objective_weights = {'secretion': 0.01, 'efficiency': 0.000001, 'kinetics': 0.00001}

In [34]:
# objective_weights = {'secretion': 0.01, 'efficiency': 0.000001, 'kinetics': 0.00001}
solution_flux, test_reaction_names, S_new, test_metabolites, test_kinetic = test_NetworkFlowModel(
                                            objective_weights,
                                            uptake_addition=uptake_addition,  uptake_removal=uptake_removal,
                                            remove_reaction=remove_reaction,
)

sim_flux = pd.DataFrame({
    'flux': solution_flux,
    'is_new': [
        'New Reactions' if id in fba_new_reaction_ids 
        else 'TEMP' if id in add_reaction.keys() 
        else 'Old Reactions' 
            for id in test_reaction_names
    ]
}, index=test_reaction_names)

met_of_interest = ['CPD-18720[e]', 'CPD-18720[p]', 'CPD-18720[c]', 'D-SORBITOL-6-P[c]']
S_new = pd.DataFrame(S_new, index=test_metabolites, columns=test_reaction_names)
S_met, rxns  = get_subset_S(S_new, met_of_interest)
rxns
rxn_flux = sim_flux.loc[rxns]
rxn_flux['kinetic'] = [test_kinetic[r] if r in kinetic_reaction_ids else False for r in rxn_flux.index]
rxn_flux

Force Reaction ID is: None
Optimal objective function reached is: 24198.44831166592


,flux,is_new,kinetic
RXN-14812-CPD-18720//FRUCTOSE-6P.23. (reverse),-0.00000,Old Reactions,False
RXN0-7189-D-SORBITOL-6-P//D-SORBITOL-6-P.31.,-0.00000,Old Reactions,False
RXN0-7189-D-SORBITOL-6-P//D-SORBITOL-6-P.31. (reverse),-0.00000,Old Reactions,False
RXN0-7208-CPD-18720//CPD-18720.21.,0.00000,Old Reactions,False
RXN0-7208-CPD-18720//CPD-18720.21. (reverse),388790.87963,Old Reactions,False
RXN0-7209-CPD-18720//CPD-18720.21.,0.00000,Old Reactions,False
RXN0-7209-CPD-18720//CPD-18720.21. (reverse),0.00000,Old Reactions,False
SORB6PDEHYDROG-RXN,-0.00000,Old Reactions,False
SORB6PDEHYDROG-RXN (reverse),-0.00000,Old Reactions,False
SORB6PDEHYDROG-RXN-D-SORBITOL-6-P/NAD//CPD-18720/NADH/PROTON.42.,-0.00000,Old Reactions,False


I don't see how SORBITOL-6-PHOSPHATASE-RXN would be favored unless we knockout a bunch of fructose conversion equations. It is also not kinetically inclined.

##  MANNITOL-1-PHOSPHATASE-RXN

"MANNITOL-1-PHOSPHATASE-RXN"	{"MANNITOL-1P[CCO-CYTOSOL]": -1, "WATER[CCO-CYTOSOL]": -1, "MANNITOL[CCO-CYTOSOL]": 1, "Pi[CCO-CYTOSOL]": 1}

In [36]:
met_of_interest = ['MANNITOL-1P[c]']
S_new = pd.DataFrame(S_new, index=test_metabolites, columns=test_reaction_names)
S_met, rxns  = get_subset_S(S_new, met_of_interest)
rxns
rxn_flux = sim_flux.loc[rxns]
rxn_flux['kinetic'] = [test_kinetic[r] if r in kinetic_reaction_ids else False for r in rxn_flux.index]
rxn_flux

,flux,is_new,kinetic
MANNITOL-1-PHOSPHATASE-RXN,-0.0,New Reactions,False
MANNPDEHYDROG-RXN,61039.0,Old Reactions,61917.0
MANNPDEHYDROG-RXN (reverse),61039.0,Old Reactions,61039.0
RXN0-7189-MANNITOL-1P//MANNITOL-1P.25.,-0.0,Old Reactions,False
RXN0-7189-MANNITOL-1P//MANNITOL-1P.25. (reverse),-0.0,Old Reactions,False
TRANS-RXN-156,0.0,New Reactions,False


## 3.1.3.68-RXN

In [40]:
met_of_interest = ['2-DEOXY-D-GLUCOSE-6-PHOSPHATE[c]']
S_new = pd.DataFrame(S_new, index=test_metabolites, columns=test_reaction_names)
S_met, rxns  = get_subset_S(S_new, met_of_interest)
rxns
rxn_flux = sim_flux.loc[rxns,:]
rxn_flux['kinetic'] = [test_kinetic[r] if r in kinetic_reaction_ids else False for r in rxn_flux.index]
rxn_flux

,flux,is_new,kinetic
3.1.3.68-RXN[CCO-CYTOSOL]-2-DEOXY-D-GLUCOSE-6-PHOSPHATE/WATER//2-DEOXY-D-GLUCOSE/Pi.71.,-0.0,New Reactions,False
3.1.3.68-RXN[CCO-CYTOSOL]-2-DEOXY-D-GLUCOSE-6-PHOSPHATE/WATER//2-DEOXY-D-GLUCOSE/Pi.71.__G6932-MONOMER,-0.0,New Reactions,30553.0
TRANS-RXN0-540,-0.0,Old Reactions,False


In [41]:
metabolism_basal.allowed_exchange_uptake.copy()

{'AMMONIUM[c]',
 'CA+2[p]',
 'CARBON-DIOXIDE[p]',
 'CL-[p]',
 'CO+2[p]',
 'FE+2[p]',
 'FE+3[p]',
 'GLC[p]',
 'K+[p]',
 'L-SELENOCYSTEINE[c]',
 'MG+2[p]',
 'MN+2[p]',
 'NA+[p]',
 'NI+2[p]',
 'OXYGEN-MOLECULE[p]',
 'Pi[p]',
 'SULFATE[p]',
 'WATER[p]',
 'ZN+2[p]'}

## 2-DEOXY-D-GLUCOSE can be used as sole Phosphate source in E. coli media (MIX0-694)
TODO: Simulate MIX0-694 media https://ecocyc.org/ECOLI/NEW-IMAGE?type=Growth-Media&object=MIX0-694

TODO2: missing 2-DEOXY-D-GLUCOSE-6-PHOSPHATE decomposition pathway from Metacyc https://biocyc.org/pathway?orgid=META&id=PWY-8121. add reactions

In [52]:
uptake_addition = set([
                       # 'CPD0-2459[e]',
                       'PYRUVATE[e]',
                       '2-DEOXY-D-GLUCOSE-6-PHOSPHATE[e]'
                       ])
uptake_removal = set([
                      'Pi[p]',
                      'GLC[p]',
                      'CA+2[p]'
                     ])

add_reaction = {
    'TEMP-RXN-18514': {
        '2-DEOXY-D-GLUCOSE[c]': -1,
        'OXYGEN-MOLECULE[c]': -1,
        'CPD-19841[c]': 1,
        'HYDROGEN-PEROXIDE[c]': 1,
    },
    'TEMP-RXN-18513': {
        'CPD-19481[c]': -1,
        'WATER[c]':-1,
        'DE-O-GLUCONATE[c]':1,
        'PROTON[c]':1,
    }
}
new_exchange_molecules = set(['2-DEOXY-D-GLUCOSE-6-PHOSPHATE[e]']) 

objective_weights = {'secretion': 0.01, 'efficiency': 0.000001, 'kinetics': 0.000001}

In [54]:
solution_flux, test_reaction_names, S_new, test_metabolites, test_kinetic = test_NetworkFlowModel(
                                            objective_weights,
                                            uptake_addition=uptake_addition,  uptake_removal=uptake_removal,
)

sim_flux = pd.DataFrame({
    'flux': solution_flux,
    'is_new': [
        'New Reactions' if id in fba_new_reaction_ids 
        else 'TEMP' if id in add_reaction.keys() 
        else 'Old Reactions' 
            for id in test_reaction_names
    ]
}, index=test_reaction_names)

met_of_interest = ['2-DEOXY-D-GLUCOSE-6-PHOSPHATE[e]', '2-DEOXY-D-GLUCOSE-6-PHOSPHATE[p]', '2-DEOXY-D-GLUCOSE-6-PHOSPHATE[c]', 'PYRUVATE[e]']
S_new = pd.DataFrame(S_new, index=test_metabolites, columns=test_reaction_names)
S_met, rxns  = get_subset_S(S_new, met_of_interest)
rxns
rxn_flux = sim_flux.loc[rxns]
rxn_flux['kinetic'] = [test_kinetic[r] if r in kinetic_reaction_ids else False for r in rxn_flux.index]
rxn_flux

Force Reaction ID is: None
Optimal objective function reached is: 31558.365876949447


,flux,is_new,kinetic
3.1.3.68-RXN[CCO-CYTOSOL]-2-DEOXY-D-GLUCOSE-6-PHOSPHATE/WATER//2-DEOXY-D-GLUCOSE/Pi.71.,-0.0,New Reactions,False
3.1.3.68-RXN[CCO-CYTOSOL]-2-DEOXY-D-GLUCOSE-6-PHOSPHATE/WATER//2-DEOXY-D-GLUCOSE/Pi.71.__G6932-MONOMER,-0.0,New Reactions,30553.0
3.1.3.68-RXN[CCO-PERI-BAC]-2-DEOXY-D-GLUCOSE-6-PHOSPHATE/WATER//2-DEOXY-D-GLUCOSE/Pi.72.,-0.0,Old Reactions,False
RXN0-7208-2-DEOXY-D-GLUCOSE-6-PHOSPHATE//2-DEOXY-D-GLUCOSE-6-PHOSPHATE.61.,0.0,Old Reactions,False
RXN0-7208-2-DEOXY-D-GLUCOSE-6-PHOSPHATE//2-DEOXY-D-GLUCOSE-6-PHOSPHATE.61. (reverse),0.0,Old Reactions,False
RXN0-7209-2-DEOXY-D-GLUCOSE-6-PHOSPHATE//2-DEOXY-D-GLUCOSE-6-PHOSPHATE.61.,0.0,Old Reactions,False
RXN0-7209-2-DEOXY-D-GLUCOSE-6-PHOSPHATE//2-DEOXY-D-GLUCOSE-6-PHOSPHATE.61. (reverse),0.0,Old Reactions,False
TRANS-RXN0-540,-0.0,Old Reactions,False
TRANS-RXN0-606-PYRUVATE//PYRUVATE.19.,-0.0,Old Reactions,False
TRANS-RXN0-606-PYRUVATE//PYRUVATE.19. (reverse),1085989.0,Old Reactions,False


In [48]:
for met in metabolism_basal.metabolite_names:
    if 'CPD0-2459' in met:
        print(met)

# EG10722 --> RXN-17955 and RXN0-6733 (kinetic)

In [101]:
# needs CPD0-1068 in enviornment
uptake_addition = set(['CPD0-1068[e]'])
add_reaction = {
    'TEMP-METHANE-TRANSPORT':{
        'CH4[p]':-1,
        'CH4[e]':1
    },
    'TEMP-METHANE-TRANSPORT (reverse)':{
        'CH4[p]':1,
        'CH4[e]':-1
    },
    'TEMP-PRPP-TRANSPORT':{
        'PRPP[c]': -1,
        'PRPP[p]':-1,
    },
    'TEMP-PRPP-TRANSPORT2':{
        'PRPP[p]': -1,
        'PRPP[e]':-1,
    },
    
}
add_metabolite = ['CH4[e]']
force_reaction = ['RXN-13859']

In [107]:
objective_weights = {'secretion': 0.01, 'efficiency': 0.000001, 'kinetics': 0.001}
solution_flux, test_reaction_names, S_new, test_metabolites, test_kinetic = test_NetworkFlowModel(
                                            objective_weights,
                                            uptake_addition=uptake_addition, add_reaction=add_reaction, add_metabolite=add_metabolite, force_reaction=force_reaction,
)

sim_flux = pd.DataFrame({
    'flux': solution_flux,
    'is_new': [
        'New Reactions' if id in fba_new_reaction_ids 
        else 'TEMP' if id in add_reaction.keys() 
        else 'Old Reactions' 
            for id in test_reaction_names
    ]
}, index=test_reaction_names)

met_of_interest = ['CPD0-1068[p]','CH4[p]', 'ADENINE[c]']
S_new = pd.DataFrame(S_new, index=test_metabolites, columns=test_reaction_names)
S_met, rxns  = get_subset_S(S_new, met_of_interest)
rxns
rxn_flux = sim_flux.loc[rxns]
rxn_flux['kinetic'] = [test_kinetic[r] if r in kinetic_reaction_ids else False for r in rxn_flux.index]
rxn_flux

(6077, 9358)
(6077, 9362)
Force Reaction ID is: [1934]
Optimal objective function reached is: 39545.76636508989


,flux,is_new,kinetic
2.7.8.25-RXN,-0.0,New Reactions,False
5-METHYLTHIOADENOSINE-PHOSPHORYLASE-RXN,-0.0,Old Reactions,False
ABC-23-RXN-ATP/CPD0-1068/WATER//ADP/CPD0-1068/Pi/PROTON.45.,-0.0,Old Reactions,False
ADENINE-DEAMINASE-RXN,0.0,Old Reactions,142.0
ADENOSINE-NUCLEOSIDASE-RXN,-0.0,Old Reactions,False
ADENOSYLHOMOCYSTEINE-NUCLEOSIDASE-RXN,93.0,Old Reactions,1999.0
ADENPHOSPHOR-RXN,-0.0,Old Reactions,False
ADENPHOSPHOR-RXN (reverse),2.0,Old Reactions,False
ADENPHOSPHOR-RXN__DEOD-CPLX,2.0,Old Reactions,2.0
ADENPRIBOSYLTRAN-RXN (reverse),585.0,Old Reactions,False


In [60]:
met_of_interest = ['NIACINAMIDE[c]']
S_new = pd.DataFrame(S_new, index=test_metabolites, columns=test_reaction_names)
S_met, rxns  = get_subset_S(S_new, met_of_interest)

In [76]:
temp = S.loc[:, rxns]
a = temp.loc[~np.all(temp==0, axis=1), :]
a

,NICOTINAMID-RXN,NMNNUCLEOSID-RXN,NMNNUCLEOSID-RXN-NICOTINAMIDE_NUCLEOTIDE/WATER//CPD-15318/NIACINAMIDE/PROTON.60.,NMNNUCLEOSID-RXN-NICOTINAMIDE_NUCLEOTIDE/WATER//CPD-16551/NIACINAMIDE/PROTON.60.,RXN-13859,RXN-19919,RXN-22964,RXN0-6445,RXN0-7078,RXN0-7092 (reverse)
ADENOSINE_DIPHOSPHATE_RIBOSE[c],0,0,0,0,1,0,0,0,0,0
AMMONIUM[c],1,0,0,0,0,0,0,0,0,0
CHEY-MONOMER[c],0,0,0,0,0,0,0,1,0,0
CPD-14762[c],0,0,0,0,0,0,0,1,1,0
CPD-15317[c],0,1,0,0,0,0,0,0,0,0
CPD-15318[c],0,0,1,0,0,0,0,0,0,0
CPD-16551[c],0,0,0,1,0,0,0,0,0,0
CPD-21533[c],0,0,0,0,0,1,0,0,0,0
CPD-25444[c],0,0,0,0,0,0,1,0,0,0
CPD3O-0[c],0,0,0,0,0,-1,0,0,-1,0


In [94]:
met_of_interest = ['NICOTINAMIDE_NUCLEOTIDE[c]', '']
S_new = pd.DataFrame(S_new, index=test_metabolites, columns=test_reaction_names)
S_met, rxns  = get_subset_S(S_new, met_of_interest)
rxns
rxn_flux = sim_flux.loc[rxns]
rxn_flux['kinetic'] = [test_kinetic[r] if r in kinetic_reaction_ids else False for r in rxn_flux.index]
rxn_flux

,flux,is_new,kinetic
2.7.7.1-RXN,0.0,Old Reactions,False
DNA-LIGASE-NAD+-RXN,-0.0,Old Reactions,False
NADPYROPHOSPHAT-RXN,-0.0,Old Reactions,False
NMNAMIDOHYDRO-RXN,-0.0,Old Reactions,113.0
NMNNUCLEOSID-RXN,-0.0,Old Reactions,False
NMNNUCLEOSID-RXN-NICOTINAMIDE_NUCLEOTIDE/WATER//CPD-15318/NIACINAMIDE/PROTON.60.,-0.0,Old Reactions,False
NMNNUCLEOSID-RXN-NICOTINAMIDE_NUCLEOTIDE/WATER//CPD-16551/NIACINAMIDE/PROTON.60.,-0.0,Old Reactions,False
RIBOSYLNICOTINAMIDE-KINASE-RXN,-0.0,Old Reactions,False
RXN-17920,-0.0,Old Reactions,False
RXN0-7346,-0.0,Old Reactions,False


1-2-Diglycerides[c]                         False
1-AMINO-PROPAN-2-ONE-3-PHOSPHATE[c]         False
1-Cys-Peroxiredoxin-L-cysteine[c]           False
1-Cys-Peroxiredoxin-L-hydroxycysteine[c]    False
1-KETO-2-METHYLVALERATE[c]                  False
                                            ...  
tRNAs-with-glutamylated-queuosine[c]        False
tRNAs-with-queuine[c]                       False
tRNAs[c]                                    False
trans-delta2-arachidoyl-ACPs[c]             False
type-IV-prepillin[c]                        False
Length: 6076, dtype: bool

In [78]:
metabolism_basal.exchange_molecules.copy()

{'ACET[p]',
 'AMMONIUM[c]',
 'BETAINE[p]',
 'BUTANAL[c]',
 'CA+2[p]',
 'CARBON-DIOXIDE[p]',
 'CARBON-MONOXIDE[p]',
 'CH33ADO[c]',
 'CL-[p]',
 'CO+2[p]',
 'CPD-10774[p]',
 'CPD-108[p]',
 'CPD-1772[c]',
 'CPD-239[p]',
 'CPD-534[p]',
 'CPD-560[p]',
 'CPD-564[c]',
 'CPD0-2167[c]',
 'D-ALANINE[p]',
 'D-LACTATE[p]',
 'ETOH[p]',
 'FE+2[p]',
 'FE+3[p]',
 'FORMATE[p]',
 'GLC[p]',
 'GLYCEROL[p]',
 'GLYCOLALDEHYDE[c]',
 'GLYCOLLATE[c]',
 'HYDROGEN-MOLECULE[c]',
 'HYPOXANTHINE[p]',
 'INDOLE[p]',
 'K+[p]',
 'L-SELENOCYSTEINE[c]',
 'METOH[p]',
 'MG+2[p]',
 'MI-PENTAKISPHOSPHATE[p]',
 'MN+2[p]',
 'NA+[p]',
 'NI+2[p]',
 'OXYGEN-MOLECULE[p]',
 'PROTON[p]',
 'Pi[p]',
 'S-ADENOSYL-4-METHYLTHIO-2-OXOBUTANOATE[c]',
 'SUC[p]',
 'SULFATE[p]',
 'THYMINE[p]',
 'UNDECAPRENYL-DIPHOSPHATE[p]',
 'URACIL[p]',
 'UREA[p]',
 'WATER[p]',
 'XANTHINE[p]',
 'ZN+2[p]'}